Optuna is a smarter, Bayesian Optimization-based method that finds optimal hyperparameters quickly.

✅ Pros: Faster and more efficient than Grid/Random Search.
❌ Cons: Slightly more complex to set up.

In [31]:
import numpy as np

import optuna

from xgboost import XGBClassifier

from sklearn.datasets import make_classification

from sklearn.model_selection import cross_val_score, train_test_split

from sklearn.metrics import accuracy_score,classification_report

In [32]:
# Generate synthetic data

X,y = make_classification(n_samples=500, n_features=10, n_informative=2, n_redundant=2, random_state=42)

In [33]:
# Split the data

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size= 0.3, random_state= 42)

In [34]:
# Define objective function

def objective(trial):

    params = {

        'n_estimators' : trial.suggest_int('n_estimators' , 50, 300 , step=50),
        'learning_rate' : trial.suggest_float('learning_rate' , 0.01, 0.3, log=True), # Actually its log but depreciated in newer versions so use float with log
        'max_depth' : trial.suggest_int('max_depth' , 3, 10),
        'subsample' : trial.suggest_float('subsample' , 0.5, 1.0),
        'colsample_bytree' : trial.suggest_float('colsample_bytree' , 0.5, 1.0)
    }
   
    # Initialize the model
    
    model = XGBClassifier(**params, eval_metric = 'logloss')

    model.fit(X_train,y_train)

    # Perform 5-fold cross-validation and return mean accuracy (to avoid overfitting if you use only X_train and y_train)

    score = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy').mean()
    
    return score  # Higher accuracy is better


In [35]:
# Run Optuna

study = optuna.create_study(direction='maximize')

study.optimize(objective, n_trials =20)

[I 2025-01-30 12:58:29,802] A new study created in memory with name: no-name-86f12bdf-b1ad-472e-a40a-d300127d5f7f
[I 2025-01-30 12:58:30,318] Trial 0 finished with value: 0.9257142857142856 and parameters: {'n_estimators': 50, 'learning_rate': 0.18640794044507109, 'max_depth': 9, 'subsample': 0.5569162408556458, 'colsample_bytree': 0.5274869689898543}. Best is trial 0 with value: 0.9257142857142856.
[I 2025-01-30 12:58:32,516] Trial 1 finished with value: 0.9428571428571428 and parameters: {'n_estimators': 200, 'learning_rate': 0.018773060159296372, 'max_depth': 10, 'subsample': 0.651449306755092, 'colsample_bytree': 0.7324840258100966}. Best is trial 1 with value: 0.9428571428571428.
[I 2025-01-30 12:58:33,954] Trial 2 finished with value: 0.9314285714285715 and parameters: {'n_estimators': 200, 'learning_rate': 0.162810780615377, 'max_depth': 9, 'subsample': 0.844046841957707, 'colsample_bytree': 0.6025177492206323}. Best is trial 1 with value: 0.9428571428571428.
[I 2025-01-30 12:58

In [36]:
print("Best parameters:", study.best_params)

print("Best score:", study.best_value)

Best parameters: {'n_estimators': 200, 'learning_rate': 0.018773060159296372, 'max_depth': 10, 'subsample': 0.651449306755092, 'colsample_bytree': 0.7324840258100966}
Best score: 0.9428571428571428


In [37]:
# Evaluate the best model

best_model = XGBClassifier(**study.best_params, eval_metric = 'logloss' )

best_model.fit(X_train,y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7324840258100966, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='logloss', feature_types=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.018773060159296372,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [39]:
# Predict with model

y_pred = best_model.predict(X_test)


In [41]:
print(f"Accuracy Score : {accuracy_score(y_test,y_pred)}")

print("Classification Report :\n")

print(classification_report(y_test, y_pred))

Accuracy Score : 0.92
Classification Report :

              precision    recall  f1-score   support

           0       0.97      0.87      0.92        79
           1       0.87      0.97      0.92        71

    accuracy                           0.92       150
   macro avg       0.92      0.92      0.92       150
weighted avg       0.93      0.92      0.92       150

